In [15]:
# %pip install openpyxl

In [16]:
import pandas as pd
import numpy as np
import os
import yfinance as yf

In [17]:
# latest_from_clipboard = pd.read_clipboard()
# latest_from_clipboard.sort_values("timestamp",ascending=False).head(30)

In [18]:
old_date = "2024-01-05"
new_date = "2024-01-08"
sheet_name = "20231210_"+old_date.replace("-","")

In [19]:
latest_from_clipboard = pd.read_excel(f"https://docs.google.com/spreadsheets/d/1-YwqS8sBO6sMMRRJV9UIKelAcPtZO6obqEKepCZ9GSU/export?exportFormat=xlsx",sheet_name=sheet_name)
print(latest_from_clipboard)

                      timestamp   price short_name
0     2023-12-11 09:00:00+03:00  254.25      THYAO
1     2023-12-11 10:00:00+03:00  251.25      THYAO
2     2023-12-11 11:00:00+03:00  250.75      THYAO
3     2023-12-11 12:00:00+03:00  249.30      THYAO
4     2023-12-11 13:00:00+03:00  248.40      THYAO
...                         ...     ...        ...
5695  2024-01-05 14:00:00+03:00   19.73      YKBNK
5696  2024-01-05 15:00:00+03:00   19.82      YKBNK
5697  2024-01-05 16:00:00+03:00   19.97      YKBNK
5698  2024-01-05 17:00:00+03:00   19.99      YKBNK
5699  2024-01-05 18:00:00+03:00   20.00      YKBNK

[5700 rows x 3 columns]


In [20]:
# new_sheet_name = "20231210_"+new_date.replace("-","")
# new_day_actual = pd.read_excel(f"https://docs.google.com/spreadsheets/d/1-YwqS8sBO6sMMRRJV9UIKelAcPtZO6obqEKepCZ9GSU/export?exportFormat=xlsx",sheet_name=new_sheet_name)
# new_day_actual = new_day_actual[new_day_actual["timestamp"] >= new_date].sort_values(["short_name","timestamp"]).reset_index(drop=True)
# new_day_actual

In [21]:
data_file_paths = ["data/"+e for e in os.listdir("data")]
# data_dict = {}
raw_data = []
for path in data_file_paths:
    # data_dict[path.split("_")[-2]] = pd.read_csv(path)
    raw_data.append(pd.read_csv(path))
raw_data = pd.concat(raw_data,axis=0)
raw_data = pd.concat([raw_data,latest_from_clipboard],axis=0)

In [22]:
data_all_indices = pd.DataFrame()
short_name = raw_data[["short_name"]].drop_duplicates()
short_name["key"] = 0
dates = sorted(list(set([e[:10] for e in raw_data["timestamp"]])))
hours = sorted(list(set([e[10:] for e in raw_data["timestamp"]])))
timestamp = pd.DataFrame({"timestamp":[e1+e2 for e1 in dates for e2 in hours],"key":0})

# timestamp = raw_data[["timestamp"]].drop_duplicates()
# timestamp["key"] = 0

data_all_indices = pd.merge(short_name,timestamp,how="outer").drop("key",axis=1)


# new_day_indices = data_all_indices[data_all_indices["timestamp"]>old_date].copy()
# new_day_indices["timestamp"] = [e.replace(old_date,new_date) for e in new_day_indices["timestamp"]]

# data_all_indices = pd.concat([data_all_indices,new_day_indices],axis = 0)

In [23]:
# interest_rate_avg = pd.read_csv("additional_data/avgint.csv")
# interest_rate_avg["date"] = [e[6:10]+"-"+e[3:5]+"-"+e[:2]+" 09:00:00+03:00" for e in interest_rate_avg["date"]]
# interest_rate_avg = interest_rate_avg.rename(columns={"date":"timestamp","AvgInt":"avg_int"})
# interest_rate_avg["timestamp"] = (pd.to_datetime(interest_rate_avg["timestamp"])+pd.DateOffset(days=3)).astype(str)
# interest_rate_avg["avg_int"] = pd.to_numeric(interest_rate_avg["avg_int"],errors="coerce")
# interest_rate_avg[interest_rate_avg["timestamp"] < new_date]

In [24]:
tickers = raw_data["short_name"].drop_duplicates().sort_values().values
daily_stock = yf.download(tickers=[t+".IS" for t in tickers], period="max", interval='1d').reset_index()
daily_stock = daily_stock[["Date","Volume"]]
daily_stock.columns = [e[1][:-3] if e[1] != "" else "timestamp" for e in daily_stock.columns]
daily_stock["timestamp"] = [e+" 09:00:00+03:00" for e in daily_stock["timestamp"].astype(str)]
daily_stock = daily_stock.melt(id_vars="timestamp",var_name="short_name",value_name="volume")
daily_stock

[*********************100%%**********************]  30 of 30 completed

1 Failed download:
['KCHOL.IS']: OperationalError('database is locked')


,timestamp,short_name,volume
0,2000-05-10 09:00:00+03:00,AKBNK,1284067.0
1,2000-05-11 09:00:00+03:00,AKBNK,608846.0
2,2000-05-12 09:00:00+03:00,AKBNK,2283592.0
3,2000-05-15 09:00:00+03:00,AKBNK,122956.0
4,2000-05-16 09:00:00+03:00,AKBNK,1331524.0
...,...,...,...
183025,2023-12-29 09:00:00+03:00,YKBNK,132638201.0
183026,2024-01-02 09:00:00+03:00,YKBNK,164641359.0
183027,2024-01-03 09:00:00+03:00,YKBNK,153558031.0
183028,2024-01-04 09:00:00+03:00,YKBNK,211488897.0


In [25]:
usd_daily = yf.download(tickers="TRY=X", period="max", interval='1d').reset_index()
usd_daily.columns = ["timestamp","usd_open","usd_high","usd_low","usd_close","adj_close","vol"]
usd_daily["timestamp"] = (pd.to_datetime((usd_daily["timestamp"]))+pd.DateOffset(days=1)).astype(str)
usd_daily["timestamp"] = [e+" 09:00:00+03:00" for e in usd_daily["timestamp"]]
usd_daily = usd_daily.drop(["adj_close","vol"],axis=1)
# usd_daily = usd_daily[usd_daily["timestamp"] < new_date]
usd_daily["usd_daily_change"] = usd_daily["usd_close"]/usd_daily["usd_open"]-1
usd_daily["usd_weekly_change"] = usd_daily["usd_close"]/usd_daily["usd_close"].shift(7)-1
usd_daily["usd_monthly_change"] = usd_daily["usd_close"]/usd_daily["usd_close"].shift(28)-1
usd_daily = usd_daily[["timestamp","usd_close","usd_daily_change","usd_weekly_change","usd_monthly_change"]]

[*********************100%%**********************]  1 of 1 completed


In [26]:
data = raw_data.copy()
data = data.groupby(["short_name","timestamp"]).median().reset_index()
data = pd.merge(data_all_indices,data,"outer")

# data = data[data["timestamp"] >= "2021"]

data = data.sort_values(["timestamp","short_name"]).reset_index(drop=True)

# data = data.merge(interest_rate_avg,how="left")
# data["avg_int"] = data.groupby(["short_name"])["avg_int"].ffill()

data = data.merge(usd_daily,how="left")
data["usd_close"] = data.groupby(["short_name"])["usd_close"].ffill()
data["usd_daily_change"] = data.groupby(["short_name"])["usd_daily_change"].ffill()
data["usd_weekly_change"] = data.groupby(["short_name"])["usd_weekly_change"].ffill()
data["usd_monthly_change"] = data.groupby(["short_name"])["usd_monthly_change"].ffill()

data = data.merge(daily_stock,how="left")
data.loc[data["volume"] == 0,"volume"] = np.nan
data["volume"] = data.groupby(["short_name"])["volume"].ffill()



data["timestamp"] = pd.to_datetime(data["timestamp"])
data["year"] = data["timestamp"].dt.year
data["monthday"] = data["timestamp"].dt.day
data["weekday"] = data["timestamp"].dt.weekday
data["month"] = data["timestamp"].dt.month
# data["hour"] = data["timestamp"].dt.hour
data["hour"] = data.groupby(["short_name","year","month","monthday"])['timestamp'].transform("rank").astype(int)

# data["price"] = data.groupby(["short_name"])["price"].ffill()
# data["price"] = data.groupby(["short_name"])["price"].apply(lambda x:x.interpolate()).ffill()
data["price"] = data.groupby(["short_name"])["price"].apply(lambda x:x.interpolate()).droplevel(0).sort_index()
data["price"] = data.groupby(["short_name"])["price"].ffill()

data["prev_close"] = data.groupby(["short_name","year","month","monthday"])['price'].transform("last")#.shift(30)
data["prev_close"] = data.groupby(["short_name"])["prev_close"].shift()
data["prev_close"][data["hour"] != 1] = np.nan
data["prev_close"] = data.groupby(["short_name"])["prev_close"].ffill()
# data["prev_close"] = data.groupby(["short_name","year","month","monthday"])['price'].transform("last").shift(30)
# data["prev_close"][data["hour"] != 1] = np.nan
# data["prev_close"] = data.groupby(["short_name"])["prev_close"].ffill()
# data["change"] = data["price"]/data.groupby(["short_name"])["price"].shift()-1
data["change"] = data["price"]/data["prev_close"]-1
data["change"] = data["change"].fillna(0)

for i in range(12):
    data[f"change_lag_prev_{str(i+1).zfill(2)}"] = data.groupby(["short_name"])["change"].shift(11+10*i)
    data[f"change_lag_exact_{str(i+1).zfill(2)}"] = data.groupby(["short_name"])["change"].shift(10+10*i)
    data[f"change_lag_next_{str(i+1).zfill(2)}"] = data.groupby(["short_name"])["change"].shift(9+10*i)
    
    # data[f"change_lag2_prev_{str(i+1).zfill(2)}"] = data.groupby(["short_name"])["change"].shift(12+10*i)
    # data[f"change_lag2_next_{str(i+1).zfill(2)}"] = data.groupby(["short_name"])["change"].shift(8+10*i)

data[f"change_lag_exact_2weeks"] = data.groupby(["short_name"])["change"].shift(10*7*2)
data[f"change_lag_exact_3weeks"] = data.groupby(["short_name"])["change"].shift(10*7*3)
data[f"change_lag_exact_4weeks"] = data.groupby(["short_name"])["change"].shift(10*7*4)
data[f"change_lag_exact_5weeks"] = data.groupby(["short_name"])["change"].shift(10*7*5)
data[f"change_lag_exact_6weeks"] = data.groupby(["short_name"])["change"].shift(10*7*6)
data[f"change_lag_exact_7weeks"] = data.groupby(["short_name"])["change"].shift(10*7*7)

data["volume_try"] = data["volume"]*data.groupby(["short_name","year","month","monthday"])['price'].transform("mean")
data["volume_try"] = data.groupby(["short_name"])["volume_try"].shift(10)
data["volume_usd"] = data["volume_try"]*data["usd_close"]
data["volume"] = data.groupby(["short_name"])["volume"].shift(10)

data["volume_try_daily_change"] = data["volume_try"]/data.groupby(["short_name"])["volume_try"].shift(10)-1
data["volume_try_weekly_change"] = data["volume_try"]/data.groupby(["short_name"])["volume_try"].shift(10*7)-1
data["volume_try_biweekly_change"] = data["volume_try"]/data.groupby(["short_name"])["volume_try"].shift(10*7*2)-1
data["volume_try_monthly_change"] = data["volume_try"]/data.groupby(["short_name"])["volume_try"].shift(10*7*4)-1

data["volume_usd_daily_change"] = data["volume_usd"]/data.groupby(["short_name"])["volume_usd"].shift(10)-1
data["volume_usd_weekly_change"] = data["volume_usd"]/data.groupby(["short_name"])["volume_usd"].shift(10*7)-1
data["volume_usd_biweekly_change"] = data["volume_usd"]/data.groupby(["short_name"])["volume_usd"].shift(10*7*2)-1
data["volume_usd_monthly_change"] = data["volume_usd"]/data.groupby(["short_name"])["volume_usd"].shift(10*7*4)-1



# data = pd.concat([data,pd.get_dummies(data[["short_name"]],"short_name")],axis=1)
data["starting_price"] = data.groupby(["short_name"])['price'].transform("first")

data["change_mean"] = np.exp(data["change"]+1)
data["change_mean"] = np.log(data.groupby("timestamp")["change_mean"].transform(np.mean))-1
data["change_mean_lag"] = data["change_mean"].shift(300)


data[f"naive_forecast_1"] = data.groupby(["short_name"])["price"].shift(10)
data[f"naive_forecast_2"] = data["prev_close"]

data

C:\Users\Yavuz\AppData\Local\Temp\ipykernel_3540\810597188.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["prev_close"][data["hour"] != 1] = np.nan
C:\Users\Yavuz\AppData\Local\Temp\ipykernel_3540\810597188.py:84: FutureWarning: The provided callable <function mean at 0x00000259A421C310> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  data["change_mean"] = np.log(data.groupby("timestamp")["change_mean"].transform(np.mean))-1


,short_name,timestamp,price,usd_close,usd_daily_change,usd_weekly_change,usd_monthly_change,volume,year,monthday,...,volume_try_monthly_change,volume_usd_daily_change,volume_usd_weekly_change,volume_usd_biweekly_change,volume_usd_monthly_change,starting_price,change_mean,change_mean_lag,naive_forecast_1,naive_forecast_2
0,AKBNK,2018-01-02 09:00:00+03:00,6.9475,3.794900,0.0,-0.007000,-0.041789,NaN,2018,2,...,NaN,NaN,NaN,NaN,NaN,6.9475,0.000000,NaN,NaN,NaN
1,ARCLK,2018-01-02 09:00:00+03:00,18.4146,3.794900,0.0,-0.007000,-0.041789,NaN,2018,2,...,NaN,NaN,NaN,NaN,NaN,18.4146,0.000000,NaN,NaN,NaN
2,ASELS,2018-01-02 09:00:00+03:00,7.9405,3.794900,0.0,-0.007000,-0.041789,NaN,2018,2,...,NaN,NaN,NaN,NaN,NaN,7.9405,0.000000,NaN,NaN,NaN
3,BIMAS,2018-01-02 09:00:00+03:00,31.8609,3.794900,0.0,-0.007000,-0.041789,NaN,2018,2,...,NaN,NaN,NaN,NaN,NaN,31.8609,0.000000,NaN,NaN,NaN
4,DOHOL,2018-01-02 09:00:00+03:00,0.7028,3.794900,0.0,-0.007000,-0.041789,NaN,2018,2,...,NaN,NaN,NaN,NaN,NaN,0.7028,0.000000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451495,TTKOM,2024-01-05 18:00:00+03:00,26.6600,29.779079,0.0,0.015956,0.032470,60513846.0,2024,5,...,1.182632,0.189658,1.911859,0.431484,1.254343,4.9603,0.009901,0.027855,26.24,26.24
451496,TUPRS,2024-01-05 18:00:00+03:00,143.8000,29.779079,0.0,0.015956,0.032470,22304645.0,2024,5,...,-0.106334,-0.000050,-0.113166,-0.075928,-0.076972,12.2392,0.009901,0.027855,143.70,143.70
451497,VAKBN,2024-01-05 18:00:00+03:00,14.3000,29.779079,0.0,0.015956,0.032470,65558154.0,2024,5,...,-0.288492,0.810976,2.065501,0.058641,-0.265115,6.6842,0.009901,0.027855,13.93,13.93
451498,VESTL,2024-01-05 18:00:00+03:00,47.2800,29.779079,0.0,0.015956,0.032470,2873843.0,2024,5,...,-0.514580,-0.267962,-0.197572,-0.374587,-0.498631,5.9926,0.009901,0.027855,45.88,45.88


In [27]:
# data.groupby(["short_name"])["price"].apply(lambda x:x.interpolate()).droplevel(0).sort_index()
# # .ffill()

In [28]:
# data.groupby(["short_name"])["price"].apply(lambda x:x.interpolate()).ffill().reset_index()

In [29]:
# corr = data[["short_name","timestamp","price"]].pivot(columns="short_name",values="price",index="timestamp").corr()#.iloc[:,1:]
# corr.index.name = "index"
# # corr = pd.DataFrame(corr.stack())
# # corr = corr.reset_index()
# corr["corr_abs"] = np.abs(corr["corr"])
# corr.columns = ["short_name","relevant_sn","corr"]
# # corr.stack().sort_values("corr_abs").reset_index().groupby(["index"])
# corr

In [30]:
data.columns.values

array(['short_name', 'timestamp', 'price', 'usd_close',
       'usd_daily_change', 'usd_weekly_change', 'usd_monthly_change',
       'volume', 'year', 'monthday', 'weekday', 'month', 'hour',
       'prev_close', 'change', 'change_lag_prev_01',
       'change_lag_exact_01', 'change_lag_next_01', 'change_lag_prev_02',
       'change_lag_exact_02', 'change_lag_next_02', 'change_lag_prev_03',
       'change_lag_exact_03', 'change_lag_next_03', 'change_lag_prev_04',
       'change_lag_exact_04', 'change_lag_next_04', 'change_lag_prev_05',
       'change_lag_exact_05', 'change_lag_next_05', 'change_lag_prev_06',
       'change_lag_exact_06', 'change_lag_next_06', 'change_lag_prev_07',
       'change_lag_exact_07', 'change_lag_next_07', 'change_lag_prev_08',
       'change_lag_exact_08', 'change_lag_next_08', 'change_lag_prev_09',
       'change_lag_exact_09', 'change_lag_next_09', 'change_lag_prev_10',
       'change_lag_exact_10', 'change_lag_next_10', 'change_lag_prev_11',
       'change_l

In [31]:
target_col = "change"
feature_cols = ['monthday', 'weekday',
       'month', 'hour', 'change_lag_prev_01',
       'change_lag_exact_01', 'change_lag_next_01', 'change_lag_prev_02',
       'change_lag_exact_02', 'change_lag_next_02', 'change_lag_prev_03',
       'change_lag_exact_03', 'change_lag_next_03', 'change_lag_prev_04',
       'change_lag_exact_04', 'change_lag_next_04', 'change_lag_prev_05',
       'change_lag_exact_05', 'change_lag_next_05', 'change_lag_prev_06',
       'change_lag_exact_06', 'change_lag_next_06', 'change_lag_prev_07',
       'change_lag_exact_07', 'change_lag_next_07', 'change_lag_prev_08',
       'change_lag_exact_08', 'change_lag_next_08', 'change_lag_prev_09',
       'change_lag_exact_09', 'change_lag_next_09', 'change_lag_prev_10',
       'change_lag_exact_10', 'change_lag_next_10',
       # 'change_lag_prev_11',
       # 'change_lag_exact_11', 'change_lag_next_11', 'change_lag_prev_12',
       # 'change_lag_exact_12', 'change_lag_next_12',
       "change_lag_exact_2weeks","change_lag_exact_3weeks","change_lag_exact_4weeks","change_lag_exact_5weeks","change_lag_exact_6weeks","change_lag_exact_7weeks",
       'starting_price',"change_mean_lag",
       # "avg_int",
       "usd_close",
       "usd_daily_change","usd_weekly_change","usd_monthly_change",
       # 'short_name_AKBNK', 'short_name_ARCLK',
       # 'short_name_ASELS', 'short_name_BIMAS', 'short_name_DOHOL',
       # 'short_name_EKGYO', 'short_name_EREGL', 'short_name_FROTO',
       # 'short_name_GARAN', 'short_name_GUBRF', 'short_name_HALKB',
       # 'short_name_ISCTR', 'short_name_KCHOL', 'short_name_KOZAA',
       # 'short_name_KOZAL', 'short_name_KRDMD', 'short_name_PETKM',
       # 'short_name_PGSUS', 'short_name_SAHOL', 'short_name_SASA',
       # 'short_name_SISE', 'short_name_TAVHL', 'short_name_TCELL',
       # 'short_name_THYAO', 'short_name_TKFEN', 'short_name_TTKOM',
       # 'short_name_TUPRS', 'short_name_VAKBN', 'short_name_VESTL',
       # 'short_name_YKBNK'
       'volume_try',
       'volume_try_daily_change',
       'volume_try_weekly_change', 'volume_try_biweekly_change',
       'volume_try_monthly_change', 
       'volume_usd',
       'volume_usd_daily_change',
       'volume_usd_weekly_change', 'volume_usd_biweekly_change',
       'volume_usd_monthly_change',
       ]
print(feature_cols)
data_for_train_all = data.dropna().reset_index(drop=True)

['monthday', 'weekday', 'month', 'hour', 'change_lag_prev_01', 'change_lag_exact_01', 'change_lag_next_01', 'change_lag_prev_02', 'change_lag_exact_02', 'change_lag_next_02', 'change_lag_prev_03', 'change_lag_exact_03', 'change_lag_next_03', 'change_lag_prev_04', 'change_lag_exact_04', 'change_lag_next_04', 'change_lag_prev_05', 'change_lag_exact_05', 'change_lag_next_05', 'change_lag_prev_06', 'change_lag_exact_06', 'change_lag_next_06', 'change_lag_prev_07', 'change_lag_exact_07', 'change_lag_next_07', 'change_lag_prev_08', 'change_lag_exact_08', 'change_lag_next_08', 'change_lag_prev_09', 'change_lag_exact_09', 'change_lag_next_09', 'change_lag_prev_10', 'change_lag_exact_10', 'change_lag_next_10', 'change_lag_exact_2weeks', 'change_lag_exact_3weeks', 'change_lag_exact_4weeks', 'change_lag_exact_5weeks', 'change_lag_exact_6weeks', 'change_lag_exact_7weeks', 'starting_price', 'change_mean_lag', 'usd_close', 'usd_daily_change', 'usd_weekly_change', 'usd_monthly_change', 'volume_try', 

In [32]:
fold_size = data_for_train_all.shape[0]//11
fold_size = (fold_size - (fold_size%300))
fold_indices = np.array([[0,(e+1)*fold_size,(e+1)*fold_size,(e+2)*fold_size] for e in range(10)])
fold_indices[-1,-1] = data_for_train_all.shape[0]
fold_indices

array([[     0,  38100,  38100,  76200],
       [     0,  76200,  76200, 114300],
       [     0, 114300, 114300, 152400],
       [     0, 152400, 152400, 190500],
       [     0, 190500, 190500, 228600],
       [     0, 228600, 228600, 266700],
       [     0, 266700, 266700, 304800],
       [     0, 304800, 304800, 342900],
       [     0, 342900, 342900, 381000],
       [     0, 381000, 381000, 422240]])

In [33]:
# data_for_train_all.iloc[fold_end_indices[-1]:].shape

In [34]:
import lightgbm as lgb
import optuna

C:\Users\Yavuz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [42]:
# models = []
# change_preds = []
def cv(trial):
    change_preds = []
    for i,(train_start,train_end,test_start,test_end) in enumerate(fold_indices[:]):
        param = {
            'verbose':-1,
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0,log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0,log=True),
            'num_leaves': trial.suggest_int('num_leaves', 2, 256),
            'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1.0),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        }
        np.random.seed(1)
        X = data_for_train_all[feature_cols].iloc[train_start:train_end,:]
        y = data_for_train_all[target_col].iloc[train_start:train_end]
        model = lgb.LGBMRegressor(**param)
        model.fit(X,y)
        X_test = data_for_train_all[feature_cols].iloc[test_start:test_end,:]
        y_test_pred = model.predict(X_test)
        # models.append(model)
        change_preds.append(y_test_pred)
        # print(i)
    change_preds_np = np.array([])
    for cp in change_preds:
        change_preds_np = np.append(change_preds_np,cp)
    change_preds_np = np.append([np.nan]*fold_size,change_preds_np)
    change_preds_np.shape

    data_for_train_all["model_change_pred"] = change_preds_np
    data_for_train_all["model_price_pred"] = (data_for_train_all["model_change_pred"]+1)*data_for_train_all["prev_close"]
    new_day_pred = data_for_train_all.iloc[:][["short_name","timestamp","model_price_pred","price"]].sort_values(["short_name","timestamp"]).reset_index(drop=True)
    new_day_pred["model_price_pred"] = new_day_pred["model_price_pred"].values.round(4)

    new_day_pred["abs_error"] = (new_day_pred["model_price_pred"]-new_day_pred["price"]).abs()
    new_day_eval_agg = new_day_pred.drop("timestamp",axis=1).groupby("short_name").sum()
    new_day_eval_agg["wmape"] = (new_day_eval_agg["abs_error"]/new_day_eval_agg["price"])
    return new_day_eval_agg.mean()["wmape"]
search = optuna.create_study(direction='minimize')
search.optimize(cv, n_trials=500)

import pickle
import joblib
joblib.dump(search,"search.pkl")
joblib.dump(search.trials_dataframe(),"search_df.pkl")
pd.set_option('display.max_rows', 1000)
search.trials_dataframe()
# cv()

[I 2024-01-07 22:10:04,867] A new study created in memory with name: no-name-1df38790-ed68-4e10-88bb-8dd8e2577819
[I 2024-01-07 22:10:41,963] Trial 0 finished with value: 0.01736100817675697 and parameters: {'reg_alpha': 1.5830962667469517e-06, 'reg_lambda': 2.3835321004414665e-07, 'num_leaves': 203, 'feature_fraction': 0.9727984208285545, 'bagging_fraction': 0.7007871826719162, 'bagging_freq': 4, 'min_child_samples': 83}. Best is trial 0 with value: 0.01736100817675697.
[I 2024-01-07 22:10:55,232] Trial 1 finished with value: 0.019673103903289465 and parameters: {'reg_alpha': 0.00010108885920725652, 'reg_lambda': 1.0389776066666256e-05, 'num_leaves': 28, 'feature_fraction': 0.6889463050736304, 'bagging_fraction': 0.21486791972758473, 'bagging_freq': 6, 'min_child_samples': 67}. Best is trial 0 with value: 0.01736100817675697.
[I 2024-01-07 22:11:09,508] Trial 2 finished with value: 0.017282259785969926 and parameters: {'reg_alpha': 8.538416223534092e-05, 'reg_lambda': 9.44840004273376

,number,value,datetime_start,datetime_complete,duration,params_bagging_fraction,params_bagging_freq,params_feature_fraction,params_min_child_samples,params_num_leaves,params_reg_alpha,params_reg_lambda,state
0,0,0.017361,2024-01-07 22:10:04.869788,2024-01-07 22:10:41.963047,0 days 00:00:37.093259,0.700787,4,0.972798,83,203,1.583096e-06,2.383532e-07,COMPLETE
1,1,0.019673,2024-01-07 22:10:41.964042,2024-01-07 22:10:55.231867,0 days 00:00:13.267825,0.214868,6,0.688946,67,28,1.010889e-04,1.038978e-05,COMPLETE
2,2,0.017282,2024-01-07 22:10:55.233868,2024-01-07 22:11:09.508750,0 days 00:00:14.274882,0.455324,1,0.198730,71,20,8.538416e-05,9.448400e-06,COMPLETE
3,3,0.016905,2024-01-07 22:11:09.509751,2024-01-07 22:11:41.283175,0 days 00:00:31.773424,0.968397,1,0.286557,84,250,4.284678e-03,1.786082e-02,COMPLETE
4,4,0.016747,2024-01-07 22:11:41.284176,2024-01-07 22:12:10.261640,0 days 00:00:28.977464,0.652476,3,0.622829,32,253,2.997735e+00,3.220535e-07,COMPLETE
5,5,0.017642,2024-01-07 22:12:10.263640,2024-01-07 22:12:27.354406,0 days 00:00:17.090766,0.606100,3,0.944746,9,30,2.083777e-02,6.626600e+00,COMPLETE
6,6,0.017615,2024-01-07 22:12:27.355406,2024-01-07 22:12:49.003660,0 days 00:00:21.648254,0.916619,6,0.576148,8,80,3.588235e-06,2.729934e-01,COMPLETE
7,7,0.016902,2024-01-07 22:12:49.005659,2024-01-07 22:13:14.055929,0 days 00:00:25.050270,0.797459,5,0.335705,92,169,3.803557e-04,9.510799e+00,COMPLETE
8,8,0.017264,2024-01-07 22:13:14.057930,2024-01-07 22:13:27.567033,0 days 00:00:13.509103,0.600148,2,0.866835,91,8,9.107306e-07,7.108355e-03,COMPLETE
9,9,0.017046,2024-01-07 22:13:27.568033,2024-01-07 22:13:58.228573,0 days 00:00:30.660540,0.897626,2,0.266779,80,221,2.561878e-04,4.186597e+00,COMPLETE


In [43]:
search.best_params

{'reg_alpha': 9.963560916189403,
 'reg_lambda': 2.0970249799778635,
 'num_leaves': 34,
 'feature_fraction': 0.7545810668686815,
 'bagging_fraction': 0.10000380384529119,
 'bagging_freq': 7,
 'min_child_samples': 7}

In [41]:
pd.set_option('display.max_rows', 1000)
search.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_bagging_fraction,params_bagging_freq,params_feature_fraction,params_min_child_samples,params_num_leaves,params_reg_alpha,params_reg_lambda,state
0,0,0.017367,2024-01-07 21:25:15.319098,2024-01-07 21:25:38.777882,0 days 00:00:23.458784,0.907999,3,0.459886,74,98,1.171148e-01,6.693132e-06,COMPLETE
1,1,0.017321,2024-01-07 21:25:38.778882,2024-01-07 21:26:17.308265,0 days 00:00:38.529383,0.975694,2,0.904260,22,186,3.927841e-03,1.643238e-06,COMPLETE
2,2,0.017546,2024-01-07 21:26:17.309265,2024-01-07 21:26:40.687807,0 days 00:00:23.378542,0.980715,6,0.881735,96,62,1.081395e-08,5.857811e-03,COMPLETE
3,3,0.018029,2024-01-07 21:26:40.689807,2024-01-07 21:27:02.310890,0 days 00:00:21.621083,0.265847,1,0.989719,26,80,8.143627e-06,6.966347e-05,COMPLETE
4,4,0.017052,2024-01-07 21:27:02.311890,2024-01-07 21:27:23.509690,0 days 00:00:21.197800,0.729983,1,0.466295,85,113,5.586763e-06,2.975546e-07,COMPLETE
5,5,0.017576,2024-01-07 21:27:23.510691,2024-01-07 21:27:48.168487,0 days 00:00:24.657796,0.728020,1,0.561464,80,107,1.319621e-01,4.094954e-01,COMPLETE
6,6,0.018245,2024-01-07 21:27:48.171494,2024-01-07 21:28:18.343493,0 days 00:00:30.171999,0.936103,5,0.688616,10,138,7.480897e-05,2.256719e-08,COMPLETE
7,7,0.017061,2024-01-07 21:28:18.344493,2024-01-07 21:28:38.331203,0 days 00:00:19.986710,0.281494,6,0.263350,84,114,7.261804e-07,2.293764e-04,COMPLETE
8,8,0.017638,2024-01-07 21:28:38.332203,2024-01-07 21:29:07.285956,0 days 00:00:28.953753,0.964632,7,0.178995,98,199,2.767198e-02,7.835559e-06,COMPLETE
9,9,0.018036,2024-01-07 21:29:07.286956,2024-01-07 21:29:37.819081,0 days 00:00:30.532125,0.599222,5,0.706877,95,175,1.802938e-06,7.728137e-03,COMPLETE


In [36]:
import pickle
import joblib
joblib.dump(search,"search.pkl")
joblib.dump(search.trials_dataframe(),"search_df.pkl")
search.trials_dataframe()


,number,value,datetime_start,datetime_complete,duration,params_bagging_fraction,params_bagging_freq,params_feature_fraction,params_min_child_samples,params_num_leaves,params_reg_alpha,params_reg_lambda,state
0,0,0.017743,2024-01-07 21:23:05.717897,2024-01-07 21:23:33.388548,0 days 00:00:27.670651,0.710342,5,0.174176,98,216,1.503953e-08,0.000612,COMPLETE
1,1,0.018640,2024-01-07 21:23:33.389547,2024-01-07 21:23:48.749147,0 days 00:00:15.359600,0.686657,7,0.601369,38,23,1.767608e-01,0.000017,COMPLETE


In [37]:
search.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_bagging_fraction,params_bagging_freq,params_feature_fraction,params_min_child_samples,params_num_leaves,params_reg_alpha,params_reg_lambda,state
0,0,0.017743,2024-01-07 21:23:05.717897,2024-01-07 21:23:33.388548,0 days 00:00:27.670651,0.710342,5,0.174176,98,216,1.503953e-08,0.000612,COMPLETE
1,1,0.018640,2024-01-07 21:23:33.389547,2024-01-07 21:23:48.749147,0 days 00:00:15.359600,0.686657,7,0.601369,38,23,1.767608e-01,0.000017,COMPLETE


In [21]:
change_preds_np = np.array([])
for cp in change_preds:
    change_preds_np = np.append(change_preds_np,cp)
change_preds_np = np.append([np.nan]*fold_size,change_preds_np)
change_preds_np.shape

data_for_train_all["model_change_pred"] = change_preds_np
data_for_train_all["model_price_pred"] = (data_for_train_all["model_change_pred"]+1)*data_for_train_all["prev_close"]

In [23]:
data_for_train_all[["short_name","price","model_price_pred"]]

,short_name,price,model_price_pred
0,AKBNK,7.3773,NaN
1,ARCLK,16.3561,NaN
2,ASELS,7.3798,NaN
3,BIMAS,31.8813,NaN
4,DOHOL,0.6057,NaN
...,...,...,...
436795,TTKOM,26.6600,26.744004
436796,TUPRS,143.8000,147.297458
436797,VAKBN,14.3000,14.175467
436798,VESTL,47.2800,46.739604


In [42]:
new_day_pred = data_for_train_all.iloc[:][["short_name","timestamp","model_price_pred","price"]].sort_values(["short_name","timestamp"]).reset_index(drop=True)
new_day_pred["model_price_pred"] = new_day_pred["model_price_pred"].values.round(4)

new_day_pred["abs_error"] = (new_day_pred["model_price_pred"]-new_day_pred["price"]).abs()
new_day_eval_agg = new_day_pred.drop("timestamp",axis=1).groupby("short_name").sum()
new_day_eval_agg["wmape"] = (new_day_eval_agg["abs_error"]/new_day_eval_agg["price"])
new_day_eval_agg.mean()["wmape"]
# new_day_pred

0.020339993934764274

In [ ]:


perf_df = data_for_train_all[["short_name","year","month","monthday","price","model_price_pred","naive_forecast_1","naive_forecast_2"]].dropna().reset_index(drop=True)
# print(np.power(perf_df["model_price_pred"]-perf_df["price"],2).mean())
# print(np.power(perf_df["naive_forecast_1"]-perf_df["price"],2).mean())
# print(np.power(perf_df["naive_forecast_2"]-perf_df["price"],2).mean())

perf_df["err_model"] = np.power(perf_df["model_price_pred"]-perf_df["price"],2)
perf_df["err_naiv1"] = np.power(perf_df["naive_forecast_1"]-perf_df["price"],2)
perf_df["err_naiv2"] = np.power(perf_df["naive_forecast_2"]-perf_df["price"],2)

perf_df.groupby(["short_name","year","month","monthday"])[["err_model","err_naiv1","err_naiv2"]].mean().reset_index()

tickers = list(set(perf_df.short_name))

# for ticker in tickers:
#     graph_data = perf_df[perf_df["short_name"] == ticker]
#     graph_data = graph_data[graph_data["year"] == 2023]
#     graph_data = graph_data[graph_data["month"] == 9]
#     graph_data[["err_model","err_naiv1","err_naiv2"]].plot()
#     plt.title(ticker)
#     plt.show()

In [601]:
from sklearn.ensemble import RandomForestRegressor

In [602]:
def custom_loss(y_true, y_pred):
    error = y_pred-y_true
    grad = error
    hess = 0 * error + 1

    return grad, hess

def custom_loss(y_true, y_pred):
    y_true_price = y_true + 1
    y_pred_price = y_pred + 1

    error = (y_pred_price-y_true_price)/(y_true_price**2)
    grad = error
    hess = 0 * error + 2/(y_true_price**2)

    return grad, hess

np.random.seed(1)
X = data_for_train_all[feature_cols].iloc[:-300,:]
y = data_for_train_all[target_col].iloc[:-300]
model = lgb.LGBMRegressor(n_jobs=-1,objective="l1")
# model = RandomForestRegressor(n_jobs=-1)

model.fit(X,y)
# sample_weight = data_for_train_all.iloc[:-300,:]["timestamp"].rank(method="dense").values
# # sample_weight = np.power(sample_weight,6)
# sample_weight = sample_weight/sample_weight.sum()
# model.fit(X,y,sample_weight=sample_weight)

X_test = data_for_train_all[feature_cols].iloc[-300:,:]
y_test_pred = model.predict(X_test)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.278353 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10515
[LightGBM] [Info] Number of data points in the train set: 434400, number of used features: 45
[LightGBM] [Info] Start training from score 0.002121


In [603]:
# y_test_pred = np.array([])
# tickers = data["short_name"].drop_duplicates().values
# for ticker in tickers:
#     print(ticker)
#     data_for_train_ticker = data_for_train_all[data_for_train_all["short_name"] == ticker]
#     np.random.seed(1)
#     X = data_for_train_ticker[feature_cols].iloc[:-10,:]
#     y = data_for_train_ticker[target_col].iloc[:-10]
#     model = lgb.LGBMRegressor(n_jobs=-1)

#     model.fit(X,y)
#     # sample_weight = data_for_train_all.iloc[:-300,:]["timestamp"].rank(method="dense").values
#     # # sample_weight = np.power(sample_weight,6)
#     # sample_weight = sample_weight/sample_weight.sum()
#     # model.fit(X,y,sample_weight=sample_weight)

#     X_test = data_for_train_ticker[feature_cols].iloc[-10:,:]
#     y_test_pred_part = model.predict(X_test)
#     y_test_pred = np.append(y_test_pred,y_test_pred_part)
# X = data_for_train_all[feature_cols].iloc[:-300,:]

In [604]:
change_preds_np = np.append([np.nan]*X.shape[0],y_test_pred)
change_preds_np.shape

(434700,)

In [605]:
data_for_train_all["model_change_pred"] = change_preds_np
data_for_train_all["model_price_pred"] = (data_for_train_all["model_change_pred"]+1)*data_for_train_all["prev_close"]

In [606]:
data_for_train_all.isna().sum()

short_name                0
timestamp                 0
price                     0
avg_int                   0
usd_close                 0
                      ...  
change_mean_lag           0
naive_forecast_1          0
naive_forecast_2          0
model_change_pred    434400
model_price_pred     434400
Length: 75, dtype: int64

In [607]:
model_pred_df = data_for_train_all.iloc[-300:][["short_name","timestamp","model_price_pred"]].pivot(columns="short_name",index="timestamp").sort_index()
model_pred_df.columns = [c[1] for c in model_pred_df.columns] 
model_pred_df
model_pred_dict = {}
for c in model_pred_df.columns:
    model_pred_dict[c] = list(model_pred_df[c].values.round(4))
print(model_pred_dict)

{'AKBNK': [38.1319, 38.1537, 38.1492, 38.1578, 38.1635, 38.1675, 38.1752, 38.1728, 38.1459, 38.067], 'ARCLK': [125.2874, 125.315, 125.4091, 125.4373, 125.4087, 125.3799, 125.3192, 125.3186, 125.2171, 125.0487], 'ASELS': [44.2702, 44.2684, 44.2747, 44.2713, 44.2899, 44.3237, 44.3099, 44.2918, 44.26, 44.1842], 'BIMAS': [301.8233, 301.8721, 301.8128, 301.7293, 301.7083, 301.6075, 301.61, 301.6325, 301.487, 301.1184], 'DOHOL': [10.5013, 10.5003, 10.5067, 10.5114, 10.5066, 10.5022, 10.5001, 10.5035, 10.5053, 10.4907], 'EKGYO': [6.9263, 6.93, 6.9294, 6.9287, 6.9276, 6.924, 6.9202, 6.9194, 6.9153, 6.9075], 'EREGL': [40.7071, 40.7302, 40.7289, 40.7289, 40.736, 40.7313, 40.73, 40.7416, 40.7223, 40.6507], 'FROTO': [729.4979, 729.5218, 729.5218, 729.6357, 729.7379, 729.3552, 729.0344, 728.5477, 728.4172, 727.5996], 'GARAN': [57.8153, 57.8132, 57.5534, 57.6627, 57.8026, 57.8309, 57.2198, 57.2885, 57.3422, 57.6829], 'GUBRF': [192.2658, 192.2988, 192.2988, 192.3421, 192.2614, 192.1788, 191.0835, 191

In [608]:
new_day_pred = data_for_train_all.iloc[-300:][["short_name","timestamp","model_price_pred"]].sort_values(["short_name","timestamp"])[["model_price_pred"]].reset_index(drop=True)
new_day_pred["model_price_pred"] = new_day_pred["model_price_pred"].values.round(4)
sheet_name = "20231210_"+new_date.replace("-","")

# print(new_day_actual)
new_day_eval = pd.concat([new_day_actual,new_day_pred],axis=1)
new_day_eval["abs_error"] = (new_day_eval["model_price_pred"]-new_day_eval["price"]).abs()
new_day_eval_agg = new_day_eval.groupby("short_name").sum().drop("timestamp",axis=1)
new_day_eval_agg["wmape"] = (new_day_eval_agg["abs_error"]/new_day_eval_agg["price"])
new_day_eval_agg.mean()

price               1086.873333
model_price_pred    1088.609407
abs_error             16.153313
wmape                  0.013981
dtype: float64

In [609]:
new_day_pred = data_for_train_all.iloc[-300:][["short_name","timestamp","model_price_pred"]].sort_values(["short_name","timestamp"])[["model_price_pred"]].reset_index(drop=True)
new_day_pred["model_price_pred"] = new_day_pred["model_price_pred"].values.round(4)

# print(new_day_actual)
new_day_eval = pd.concat([new_day_actual,new_day_pred],axis=1)
new_day_eval["abs_error"] = (new_day_eval["model_price_pred"]-new_day_eval["price"]).abs()
new_day_eval_agg = new_day_eval.groupby("short_name").sum().drop("timestamp",axis=1)
new_day_eval_agg["wmape"] = (new_day_eval_agg["abs_error"]/new_day_eval_agg["price"])
new_day_eval_agg.mean()

price               1086.873333
model_price_pred    1088.609407
abs_error             16.153313
wmape                  0.013981
dtype: float64

In [610]:
new_day_pred = data_for_train_all.iloc[-300:][["short_name","timestamp","model_price_pred"]].sort_values(["short_name","timestamp"])[["model_price_pred"]].reset_index(drop=True)
new_day_pred["model_price_pred"] = new_day_pred["model_price_pred"].values.round(4)

# print(new_day_actual)
new_day_eval = pd.concat([new_day_actual,new_day_pred],axis=1)
new_day_eval["abs_error"] = (new_day_eval["model_price_pred"]-new_day_eval["price"]).abs()
new_day_eval_agg = new_day_eval.groupby("short_name").sum().drop("timestamp",axis=1)
new_day_eval_agg["wmape"] = (new_day_eval_agg["abs_error"]/new_day_eval_agg["price"])
new_day_eval_agg.mean()

price               1086.873333
model_price_pred    1088.609407
abs_error             16.153313
wmape                  0.013981
dtype: float64

In [611]:
new_day_pred = data_for_train_all.iloc[-300:][["short_name","timestamp","model_price_pred"]].sort_values(["short_name","timestamp"])[["model_price_pred"]].reset_index(drop=True)
new_day_pred["model_price_pred"] = new_day_pred["model_price_pred"].values.round(4)

# print(new_day_actual)
new_day_eval = pd.concat([new_day_actual,new_day_pred],axis=1)
new_day_eval["abs_error"] = (new_day_eval["model_price_pred"]-new_day_eval["price"]).abs()
new_day_eval_agg = new_day_eval.groupby("short_name").sum().drop("timestamp",axis=1)
new_day_eval_agg["wmape"] = (new_day_eval_agg["abs_error"]/new_day_eval_agg["price"])
new_day_eval_agg.mean()

price               1086.873333
model_price_pred    1088.609407
abs_error             16.153313
wmape                  0.013981
dtype: float64

In [200]:
data_for_train_all.iloc[-300:][["short_name","timestamp","model_price_pred"]]

,short_name,timestamp,model_price_pred
443070,AKBNK,2023-12-26 09:00:00+03:00,38.076577
443071,ARCLK,2023-12-26 09:00:00+03:00,125.315512
443072,ASELS,2023-12-26 09:00:00+03:00,44.271683
443073,BIMAS,2023-12-26 09:00:00+03:00,301.711503
443074,DOHOL,2023-12-26 09:00:00+03:00,10.480535
...,...,...,...
443365,TTKOM,2023-12-26 18:00:00+03:00,24.917587
443366,TUPRS,2023-12-26 18:00:00+03:00,141.163780
443367,VAKBN,2023-12-26 18:00:00+03:00,12.834501
443368,VESTL,2023-12-26 18:00:00+03:00,44.497084


In [201]:
new_day_eval.groupby("short_name").mean().mean()

price               108.672500
model_price_pred    109.179144
abs_error             3.353331
dtype: float64

In [58]:
new_day_eval_agg["wmape"]

short_name
AKBNK    0.041118
ARCLK    0.004261
ASELS    0.029215
BIMAS    0.020888
DOHOL    0.027382
EKGYO    0.007277
EREGL    0.024648
FROTO    0.014037
GARAN    0.005085
GUBRF    0.155300
HALKB    0.020553
ISCTR    0.019804
KCHOL    0.018167
KOZAA    0.068403
KOZAL    0.033525
KRDMD    0.049017
PETKM    0.010643
PGSUS    0.010874
SAHOL    0.004629
SASA     0.041228
SISE     0.028766
TAVHL    0.018830
TCELL    0.044069
THYAO    0.006811
TKFEN    0.040240
TTKOM    0.013989
TUPRS    0.024493
VAKBN    0.010272
VESTL    0.021486
YKBNK    0.033750
Name: wmape, dtype: float64

In [62]:
(new_day_eval["abs_error"]/new_day_eval["price"]).mean()

0.028251881190347335

In [39]:
new_day_eval

,price,model_price_pred,abs_error
short_name,,,
AKBNK,361.98,376.863899,14.883899
ARCLK,1247.70,1251.546686,3.846686
ASELS,447.74,435.518050,12.221950
BIMAS,3104.50,3039.654183,64.845817
DOHOL,108.60,105.626332,2.973668
EKGYO,68.75,68.461288,0.288712
EREGL,415.88,405.629412,10.250588
FROTO,7468.00,7363.169193,104.830807
GARAN,563.65,562.108542,1.541458


In [30]:
(new_day_eval["model_price_pred"]-new_day_eval["price"]).abs().sum()/new_day_eval["price"].sum()

0.02302626924958063

In [24]:
(new_day_eval["model_price_pred"]-new_day_eval["price"]).abs().sum()/new_day_eval["price"].sum()

0.02302626924958063

In [30]:
model_pred_df = data_for_train_all.iloc[-300:][["short_name","timestamp","naive_forecast_2"]].pivot(columns="short_name",index="timestamp").sort_index()
model_pred_df.columns = [c[1] for c in model_pred_df.columns] 
model_pred_df
model_pred_dict = {}
for c in model_pred_df.columns:
    model_pred_dict[c] = list(model_pred_df[c].values.round(4))
print(model_pred_dict)

{'AKBNK': [35.88, 35.88, 35.88, 35.88, 35.88, 35.88, 35.88, 35.88, 35.88, 35.88], 'ARCLK': [124.0, 124.0, 124.0, 124.0, 124.0, 124.0, 124.0, 124.0, 124.0, 124.0], 'ASELS': [43.02, 43.02, 43.02, 43.02, 43.02, 43.02, 43.02, 43.02, 43.02, 43.02], 'BIMAS': [306.0, 306.0, 306.0, 306.0, 306.0, 306.0, 306.0, 306.0, 306.0, 306.0], 'DOHOL': [10.71, 10.71, 10.71, 10.71, 10.71, 10.71, 10.71, 10.71, 10.71, 10.71], 'EKGYO': [6.76, 6.76, 6.76, 6.76, 6.76, 6.76, 6.76, 6.76, 6.76, 6.76], 'EREGL': [41.12, 41.12, 41.12, 41.12, 41.12, 41.12, 41.12, 41.12, 41.12, 41.12], 'FROTO': [732.5, 732.5, 732.5, 732.5, 732.5, 732.5, 732.5, 732.5, 732.5, 732.5], 'GARAN': [55.75, 55.75, 55.75, 55.75, 55.75, 55.75, 55.75, 55.75, 55.75, 55.75], 'GUBRF': [155.4, 155.4, 155.4, 155.4, 155.4, 155.4, 155.4, 155.4, 155.4, 155.4], 'HALKB': [11.65, 11.65, 11.65, 11.65, 11.65, 11.65, 11.65, 11.65, 11.65, 11.65], 'ISCTR': [22.34, 22.34, 22.34, 22.34, 22.34, 22.34, 22.34, 22.34, 22.34, 22.34], 'KCHOL': [136.0, 136.0, 136.0, 136.0,

In [335]:
len(model_pred_dict.keys())

30

In [271]:
import matplotlib.pyplot as plt

In [272]:
data_for_train_all[(data_for_train_all["year"] == 2023) & (data_for_train_all["month"] == 11)].isna().sum()

short_name           0
timestamp            0
price                0
year                 0
monthday             0
                    ..
change_mean_lag      0
naive_forecast_1     0
naive_forecast_2     0
model_change_pred    0
model_price_pred     0
Length: 77, dtype: int64

In [273]:


perf_df = data_for_train_all[["short_name","year","month","monthday","price","model_price_pred","naive_forecast_1","naive_forecast_2"]].dropna().reset_index(drop=True)
# print(np.power(perf_df["model_price_pred"]-perf_df["price"],2).mean())
# print(np.power(perf_df["naive_forecast_1"]-perf_df["price"],2).mean())
# print(np.power(perf_df["naive_forecast_2"]-perf_df["price"],2).mean())

perf_df["err_model"] = np.power(perf_df["model_price_pred"]-perf_df["price"],2)
perf_df["err_naiv1"] = np.power(perf_df["naive_forecast_1"]-perf_df["price"],2)
perf_df["err_naiv2"] = np.power(perf_df["naive_forecast_2"]-perf_df["price"],2)

perf_df.groupby(["short_name","year","month","monthday"])[["err_model","err_naiv1","err_naiv2"]].mean().reset_index()

tickers = list(set(perf_df.short_name))

# for ticker in tickers:
#     graph_data = perf_df[perf_df["short_name"] == ticker]
#     graph_data = graph_data[graph_data["year"] == 2023]
#     graph_data = graph_data[graph_data["month"] == 9]
#     graph_data[["err_model","err_naiv1","err_naiv2"]].plot()
#     plt.title(ticker)
#     plt.show()

In [274]:
perf_df[(perf_df["year"]==2023) & (perf_df["month"]==11) & (perf_df["monthday"]==15)]

,short_name,year,month,monthday,price,model_price_pred,naive_forecast_1,naive_forecast_2,err_model,err_naiv1,err_naiv2
391980,AKBNK,2023,11,15,28.66,28.697226,28.08,28.56,0.001386,0.3364,0.0100
391981,ARCLK,2023,11,15,140.50,140.541387,136.30,139.90,0.001713,17.6400,0.3600
391982,ASELS,2023,11,15,41.48,41.483625,41.18,41.32,0.000013,0.0900,0.0256
391983,BIMAS,2023,11,15,289.75,288.361934,285.75,287.00,1.926727,16.0000,7.5625
391984,DOHOL,2023,11,15,12.96,12.961561,12.90,12.89,0.000002,0.0036,0.0049
...,...,...,...,...,...,...,...,...,...,...,...
392275,TTKOM,2023,11,15,21.54,21.436568,21.14,21.14,0.010698,0.1600,0.1600
392276,TUPRS,2023,11,15,151.70,152.901821,152.20,152.20,1.444374,0.2500,0.2500
392277,VAKBN,2023,11,15,14.97,15.334129,15.16,15.16,0.132590,0.0361,0.0361
392278,VESTL,2023,11,15,60.15,59.544525,59.30,59.30,0.366600,0.7225,0.7225


In [224]:
perf_df[(perf_df["year"]==2023) & (perf_df["month"]==11) & (perf_df["monthday"]==15)]

,short_name,year,month,monthday,price,model_price_pred,naive_forecast_1,naive_forecast_2,err_model,err_naiv1,err_naiv2
391980,AKBNK,2023,11,15,28.66,28.711491,28.08,28.56,0.002651,0.3364,0.0100
391981,ARCLK,2023,11,15,140.50,140.483734,136.30,139.90,0.000265,17.6400,0.3600
391982,ASELS,2023,11,15,41.48,41.491540,41.18,41.32,0.000133,0.0900,0.0256
391983,BIMAS,2023,11,15,289.75,287.885742,285.75,287.00,3.475459,16.0000,7.5625
391984,DOHOL,2023,11,15,12.96,12.938671,12.90,12.89,0.000455,0.0036,0.0049
...,...,...,...,...,...,...,...,...,...,...,...
392275,TTKOM,2023,11,15,21.54,21.314577,21.14,21.14,0.050816,0.1600,0.1600
392276,TUPRS,2023,11,15,151.70,153.109149,152.20,152.20,1.985700,0.2500,0.2500
392277,VAKBN,2023,11,15,14.97,15.245104,15.16,15.16,0.075682,0.0361,0.0361
392278,VESTL,2023,11,15,60.15,59.630720,59.30,59.30,0.269652,0.7225,0.7225


In [153]:
perf_df[(perf_df["year"]==2023) & (perf_df["month"]==11) & (perf_df["monthday"]==15)]

,short_name,year,month,monthday,price,model_price_pred,naive_forecast_1,naive_forecast_2,err_model,err_naiv1,err_naiv2
394080,AKBNK,2023,11,15,28.66,28.685416,28.08,28.56,0.000646,0.3364,0.0100
394081,ARCLK,2023,11,15,140.50,140.414417,136.30,139.90,0.007324,17.6400,0.3600
394082,ASELS,2023,11,15,41.48,41.467178,41.18,41.32,0.000164,0.0900,0.0256
394083,BIMAS,2023,11,15,289.75,288.279102,285.75,287.00,2.163540,16.0000,7.5625
394084,DOHOL,2023,11,15,12.96,12.940826,12.90,12.89,0.000368,0.0036,0.0049
...,...,...,...,...,...,...,...,...,...,...,...
394375,TTKOM,2023,11,15,21.54,21.336796,21.14,21.14,0.041292,0.1600,0.1600
394376,TUPRS,2023,11,15,151.70,152.925784,152.20,152.20,1.502547,0.2500,0.2500
394377,VAKBN,2023,11,15,14.97,15.256682,15.16,15.16,0.082186,0.0361,0.0361
394378,VESTL,2023,11,15,60.15,59.552259,59.30,59.30,0.357294,0.7225,0.7225


In [86]:
perf_df[(perf_df["year"]==2023) & (perf_df["month"]==11) & (perf_df["monthday"]==15)]

,short_name,year,month,monthday,price,model_price_pred,naive_forecast_1,naive_forecast_2,err_model,err_naiv1,err_naiv2
394080,AKBNK,2023,11,15,28.66,27.800363,28.08,16.29,0.738976,0.3364,153.0169
394081,ARCLK,2023,11,15,140.50,137.613269,136.30,28.60,8.333214,17.6400,12521.6100
394082,ASELS,2023,11,15,41.48,41.403885,41.18,138.50,0.005793,0.0900,9412.8804
394083,BIMAS,2023,11,15,289.75,287.343047,285.75,41.00,5.793423,16.0000,61876.5625
394084,DOHOL,2023,11,15,12.96,13.790591,12.90,290.25,0.689881,0.0036,76889.7441
...,...,...,...,...,...,...,...,...,...,...,...
394375,TTKOM,2023,11,15,21.54,21.884538,21.14,40.08,0.118706,0.1600,343.7316
394376,TUPRS,2023,11,15,151.70,152.727595,152.20,21.54,1.055951,0.2500,16941.6256
394377,VAKBN,2023,11,15,14.97,15.762148,15.16,151.70,0.627499,0.0361,18695.0929
394378,VESTL,2023,11,15,60.15,58.959875,59.30,14.97,1.416399,0.7225,2041.2324


In [75]:
perf_df

,short_name,year,month,monthday,price,model_price_pred,naive_forecast_1,naive_forecast_2,err_model,err_naiv1,err_naiv2
0,AKBNK,2018,7,30,5.2399,5.218290,5.1371,1.8131,0.000467,0.010568,11.742958
1,ARCLK,2018,7,30,11.1318,11.336671,11.1229,5.2693,0.041972,0.000079,34.368906
2,ASELS,2018,7,30,6.6198,6.648463,6.5133,11.1583,0.000822,0.011342,20.597982
3,BIMAS,2018,7,30,29.0134,29.309226,29.2415,6.6440,0.087513,0.052030,500.390056
4,DOHOL,2018,7,30,0.7917,0.868399,0.7836,29.0548,0.005883,0.000066,798.802822
...,...,...,...,...,...,...,...,...,...,...,...
402475,TTKOM,2023,12,22,25.0800,25.376559,25.4600,36.6600,0.087947,0.144400,134.096400
402476,TUPRS,2023,12,22,144.8000,152.407939,152.5000,25.0800,57.880737,59.290000,14332.878400
402477,VAKBN,2023,12,22,13.5000,13.972048,13.8700,144.8000,0.222829,0.136900,17239.690000
402478,VESTL,2023,12,22,46.1600,47.772707,47.9400,13.5000,2.600823,3.168400,1066.675600


In [276]:
perf_df.groupby(["year","month"])["err_model","err_naiv1","err_naiv2"].mean().reset_index().sort_values(["year","month"])[-24:].mean()

C:\Users\usera\AppData\Local\Temp\ipykernel_7808\2558269787.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  perf_df.groupby(["year","month"])["err_model","err_naiv1","err_naiv2"].mean().reset_index().sort_values(["year","month"])[-24:].mean()


year         2022.500000
month           6.500000
err_model      11.583181
err_naiv1      18.110767
err_naiv2      11.710205
dtype: float64

In [228]:
perf_df.groupby(["year","month"])["err_model","err_naiv1","err_naiv2"].mean().reset_index().sort_values(["year","month"])[-24:]

C:\Users\usera\AppData\Local\Temp\ipykernel_7808\235538020.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  perf_df.groupby(["year","month"])["err_model","err_naiv1","err_naiv2"].mean().reset_index().sort_values(["year","month"])[-24:]


,year,month,err_model,err_naiv1,err_naiv2
41,2022,1,1.897877,2.789613,1.785586
42,2022,2,2.772185,3.618713,2.806647
43,2022,3,1.775105,2.444819,1.716519
44,2022,4,0.774169,1.558004,0.821520
45,2022,5,1.583040,2.281383,1.622588
46,2022,6,1.489265,3.770674,1.498950
47,2022,7,1.634165,2.588392,1.742664
48,2022,8,1.736142,3.035450,1.789206
49,2022,9,3.619246,6.282644,3.680199
50,2022,10,3.871790,6.738137,4.265839


In [160]:
perf_df.groupby(["year","month"])["err_model","err_naiv1","err_naiv2"].mean().reset_index().sort_values(["year","month"])[-24:]

C:\Users\usera\AppData\Local\Temp\ipykernel_7808\235538020.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  perf_df.groupby(["year","month"])["err_model","err_naiv1","err_naiv2"].mean().reset_index().sort_values(["year","month"])[-24:]


,year,month,err_model,err_naiv1,err_naiv2
42,2022,1,1.864572,2.789613,1.785586
43,2022,2,2.642499,3.618713,2.806647
44,2022,3,1.748532,2.444819,1.716519
45,2022,4,0.765954,1.558004,0.821520
46,2022,5,1.577085,2.281383,1.622588
47,2022,6,1.506205,3.770674,1.498950
48,2022,7,1.689003,2.588392,1.742664
49,2022,8,1.749687,3.035450,1.789206
50,2022,9,3.607811,6.282644,3.680199
51,2022,10,4.038398,6.738137,4.265839


In [226]:
perf_df.groupby(["year","month","short_name"])["err_model","err_naiv1","err_naiv2"].mean().reset_index().sort_values(["year","month","short_name"]).iloc[-60:]

C:\Users\usera\AppData\Local\Temp\ipykernel_7808\960821837.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  perf_df.groupby(["year","month","short_name"])["err_model","err_naiv1","err_naiv2"].mean().reset_index().sort_values(["year","month","short_name"]).iloc[-60:]


,year,month,short_name,err_model,err_naiv1,err_naiv2
1890,2023,11,AKBNK,0.300786,0.548877,0.319135
1891,2023,11,ARCLK,4.397416,5.991545,3.705364
1892,2023,11,ASELS,0.666202,1.218674,0.682344
1893,2023,11,BIMAS,30.130094,45.777588,31.418339
1894,2023,11,DOHOL,0.058303,0.079137,0.046423
1895,2023,11,EKGYO,0.013952,0.019907,0.015891
1896,2023,11,EREGL,0.352713,0.564638,0.321375
1897,2023,11,FROTO,155.768484,285.904864,166.071364
1898,2023,11,GARAN,0.770837,1.353359,0.775905
1899,2023,11,GUBRF,35.950880,51.270580,34.189830


In [155]:
perf_df.groupby(["year","month","short_name"])["err_model","err_naiv1","err_naiv2"].mean().reset_index().sort_values(["year","month","short_name"]).iloc[-60:]

C:\Users\usera\AppData\Local\Temp\ipykernel_7808\960821837.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  perf_df.groupby(["year","month","short_name"])["err_model","err_naiv1","err_naiv2"].mean().reset_index().sort_values(["year","month","short_name"]).iloc[-60:]


,year,month,short_name,err_model,err_naiv1,err_naiv2
1920,2023,11,AKBNK,0.303033,0.548877,0.319135
1921,2023,11,ARCLK,4.294354,5.991545,3.705364
1922,2023,11,ASELS,0.678074,1.218674,0.682344
1923,2023,11,BIMAS,30.706528,45.777588,31.418339
1924,2023,11,DOHOL,0.057714,0.079137,0.046423
1925,2023,11,EKGYO,0.014042,0.019907,0.015891
1926,2023,11,EREGL,0.354812,0.564638,0.321375
1927,2023,11,FROTO,157.445865,285.904864,166.071364
1928,2023,11,GARAN,0.775352,1.353359,0.775905
1929,2023,11,GUBRF,34.475528,51.270580,34.189830


In [30]:
data_for_train_all[["year","month","short_name"]].drop_duplicates().groupby(["year","month"]).count()

short_name
year month            
2018 1              30
     2              30
     3              30
     4              30
     5              30
...                ...
2023 8              30
     9              30
     10             30
     11             30
     12             30

[72 rows x 1 columns]

In [73]:
data_for_train_all[(data_for_train_all["year"] == 2023) & (data_for_train_all["month"] == 11)].isna().sum()

short_name             0
timestamp              0
price                  0
year                   0
monthday               0
weekday                0
month                  0
hour                   0
prev_close             0
change                 0
change_lag_prev_01     0
change_lag_exact_01    0
change_lag_next_01     0
change_lag_prev_02     0
change_lag_exact_02    0
change_lag_next_02     0
change_lag_prev_03     0
change_lag_exact_03    0
change_lag_next_03     0
change_lag_prev_04     0
change_lag_exact_04    0
change_lag_next_04     0
change_lag_prev_05     0
change_lag_exact_05    0
change_lag_next_05     0
change_lag_prev_06     0
change_lag_exact_06    0
change_lag_next_06     0
change_lag_prev_07     0
change_lag_exact_07    0
change_lag_next_07     0
change_lag_prev_08     0
change_lag_exact_08    0
change_lag_next_08     0
change_lag_prev_09     0
change_lag_exact_09    0
change_lag_next_09     0
change_lag_prev_10     0
change_lag_exact_10    0
change_lag_next_10     0


In [74]:
perf_df[["year","month","short_name"]].drop_duplicates().groupby(["year","month"]).count()

short_name
year month            
2018 7              30
     8              30
     9              30
     10             30
     11             30
...                ...
2023 8              30
     9              30
     10             30
     11             30
     12             30

[66 rows x 1 columns]

In [161]:
perf_df.groupby(["year","month","short_name"])["err_naiv1","err_naiv2"].mean().reset_index().sort_values(["year","month","short_name"]).iloc[-60:]

C:\Users\usera\AppData\Local\Temp\ipykernel_7808\2739923772.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  perf_df.groupby(["year","month","short_name"])["err_naiv1","err_naiv2"].mean().reset_index().sort_values(["year","month","short_name"]).iloc[-60:]


,year,month,short_name,err_naiv1,err_naiv2
1920,2023,11,AKBNK,0.548877,0.319135
1921,2023,11,ARCLK,5.991545,3.705364
1922,2023,11,ASELS,1.218674,0.682344
1923,2023,11,BIMAS,45.777588,31.418339
1924,2023,11,DOHOL,0.079137,0.046423
1925,2023,11,EKGYO,0.019907,0.015891
1926,2023,11,EREGL,0.564638,0.321375
1927,2023,11,FROTO,285.904864,166.071364
1928,2023,11,GARAN,1.353359,0.775905
1929,2023,11,GUBRF,51.270580,34.189830


In [292]:
perf_df.groupby(["year","month","short_name"])["err_naiv1","err_naiv2"].mean().reset_index().sort_values(["year","month","short_name"]).iloc[-60:]

C:\Users\usera\AppData\Local\Temp\ipykernel_18968\2739923772.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  perf_df.groupby(["year","month","short_name"])["err_naiv1","err_naiv2"].mean().reset_index().sort_values(["year","month","short_name"]).iloc[-60:]


,year,month,short_name,err_naiv1,err_naiv2
2100,2023,11,AKBNK,0.548877,0.319135
2101,2023,11,ARCLK,5.991545,3.705364
2102,2023,11,ASELS,1.218674,0.682344
2103,2023,11,BIMAS,45.777588,31.418339
2104,2023,11,DOHOL,0.079137,0.046423
2105,2023,11,EKGYO,0.019907,0.015891
2106,2023,11,EREGL,0.564638,0.321375
2107,2023,11,FROTO,285.904864,166.071364
2108,2023,11,GARAN,1.353359,0.775905
2109,2023,11,GUBRF,51.270580,34.189830


In [284]:
last_date = raw_data["timestamp"].max()

naive_pred = raw_data[raw_data["timestamp"] == last_date].reset_index(drop=True)
naive_pred

,timestamp,price,short_name
0,2023-12-22 18:00:00+03:00,233.00,THYAO
1,2023-12-22 18:00:00+03:00,39.16,AKBNK
2,2023-12-22 18:00:00+03:00,125.70,ARCLK
3,2023-12-22 18:00:00+03:00,45.60,ASELS
4,2023-12-22 18:00:00+03:00,309.50,BIMAS
5,2023-12-22 18:00:00+03:00,11.02,DOHOL
6,2023-12-22 18:00:00+03:00,7.08,EKGYO
7,2023-12-22 18:00:00+03:00,42.00,EREGL
8,2023-12-22 18:00:00+03:00,746.50,FROTO
9,2023-12-22 18:00:00+03:00,212.90,GUBRF


In [286]:
pred = {}
for i in range(naive_pred.shape[0]):
    pred[naive_pred.iloc[i]["short_name"]] = [naive_pred.iloc[i]["price"]]*10

In [293]:
print(pred)

{'THYAO': [233.0, 233.0, 233.0, 233.0, 233.0, 233.0, 233.0, 233.0, 233.0, 233.0], 'AKBNK': [39.16, 39.16, 39.16, 39.16, 39.16, 39.16, 39.16, 39.16, 39.16, 39.16], 'ARCLK': [125.7, 125.7, 125.7, 125.7, 125.7, 125.7, 125.7, 125.7, 125.7, 125.7], 'ASELS': [45.6, 45.6, 45.6, 45.6, 45.6, 45.6, 45.6, 45.6, 45.6, 45.6], 'BIMAS': [309.5, 309.5, 309.5, 309.5, 309.5, 309.5, 309.5, 309.5, 309.5, 309.5], 'DOHOL': [11.02, 11.02, 11.02, 11.02, 11.02, 11.02, 11.02, 11.02, 11.02, 11.02], 'EKGYO': [7.08, 7.08, 7.08, 7.08, 7.08, 7.08, 7.08, 7.08, 7.08, 7.08], 'EREGL': [42.0, 42.0, 42.0, 42.0, 42.0, 42.0, 42.0, 42.0, 42.0, 42.0], 'FROTO': [746.5, 746.5, 746.5, 746.5, 746.5, 746.5, 746.5, 746.5, 746.5, 746.5], 'GUBRF': [212.9, 212.9, 212.9, 212.9, 212.9, 212.9, 212.9, 212.9, 212.9, 212.9], 'GARAN': [60.3, 60.3, 60.3, 60.3, 60.3, 60.3, 60.3, 60.3, 60.3, 60.3], 'KRDMD': [23.3, 23.3, 23.3, 23.3, 23.3, 23.3, 23.3, 23.3, 23.3, 23.3], 'KCHOL': [140.5, 140.5, 140.5, 140.5, 140.5, 140.5, 140.5, 140.5, 140.5, 140.